In [1]:
from google.colab import drive
drive.mount('./gdrive')

Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [2]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
import tensorflow as tf
# from mnist import load_mnist
mnist = tf.keras.datasets.fashion_mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()
# (x_train, t_train), (x_test, t_test) = load_mnist(one_hot_label=True)


In [3]:
#optimizer
class SGD:
    def __init__(self, lr=0.01):
        self.lr =lr

    def update(self, params, grads):
        for k in params.keys():
            params[k] -= self.lr * grads[k]

class Adam:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1 #first moment를 위한 decay
        self.beta2 = beta2 #second moment를 위한 decay
        self.iter =0
        self.m = None #first moment
        self.v = None #second moment
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v ={},{}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        self.iter +=1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0-self.beta1**self.iter)  

        for key in params.keys():
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)

In [4]:
#loss func
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


In [5]:

class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


In [6]:

class Affine:
    def __init__(self, W, b):
        self.W = W #weights 저장
        self.b = b #hidden layer 갯수만큼 bias 저장
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

#activation function
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

In [7]:
class Convolution:
    def __init__(self,W,b,stride=1,pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.col = None
        self.col_W = None

        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N,C,H,W = x.shape
        out_h = 1+ int((H+2*self.pad-FH)/self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

In [12]:

from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        #init weights, biases
        self.params={}
        self.params['W1'] = weight_init_std*np.random.randn(input_size,200)
        self.params['b1'] = np.zeros(200)
        self.params['W2']=weight_init_std*np.random.randn(200, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3']=weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        #making layer
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])
        conv = Convolution(self.params['W1'],self.params['b1'])
        self.layers['Relu1']=Relu()
        self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])
        self.layers['Relu2']=Relu()
        self.layers['Affine3'] = Affine(self.params['W3'],self.params['b3'])
        self.lastLayer=SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    def accuracy(self, x, t):
        y=self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t= np.argmax(t, axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self,x,t):
        loss_W = lambda W: self.loss(x,t)
        grads={}
        grads['W1']= numercal_gradient(loss_W, self.params['W1'])
        grads['b1']= numercal_gradient(loss_W, self.params['b1'])
        grads['W2']= numercal_gradient(loss_W, self.params['W2'])
        grads['b2']= numercal_gradient(loss_W, self.params['b2'])
        grads['W3']= numercal_gradient(loss_W, self.params['W3'])
        grads['b3']= numercal_gradient(loss_W, self.params['b3'])
        return grads

    def gradient(self,x,t):
        self.loss(x,t)

        dout=1
        dout=self.lastLayer.backward(dout)
        layers=list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads={}
        grads['W1']=self.layers['Affine1'].dW
        grads['b1']=self.layers['Affine1'].db
        grads['W2']=self.layers['Affine2'].dW
        grads['b2']=self.layers['Affine2'].db
        grads['W3']=self.layers['Affine3'].dW
        grads['b3']=self.layers['Affine3'].db
        return grads

train

In [16]:
#변수 초기화---------
network = TwoLayerNet(input_size=784, hidden_size =50,output_size=10)
iters_num=10000
train_size=x_train.shape[0]
batch_size=100
train_loss_list=[]
train_acc_list=[]
val_loss=0
test_acc_list=[]
iter_per_epoch = max(train_size/batch_size,1)
optimizer = SGD(0.0001)#(**optimizer_param)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch= t_train[batch_mask]

    #오차역전파
    grad = network.gradient(x_batch, t_batch)
    #갱신
    for key in {'W1', 'b1', 'W2', 'b2'}:
        optimizer.update(network.params,grad) #
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i%iter_per_epoch==0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train_acc : ",train_acc," / test_acc : ",test_acc)
        print("train_loss : ",sum(train_loss_list) / iter_per_epoch," / test_loss : ",network.loss(x_test,t_test))
        train_loss_list.clear()

0.003820058704070762
2.2935391591528065
train_acc :  0.10513333333333333  / test_acc :  0.1076
1.1749793251610194
0.7760331928122954
train_acc :  0.7211833333333333  / test_acc :  0.7095
0.6458890750177639
0.6562333765867484
train_acc :  0.7683666666666666  / test_acc :  0.7553
0.531925509566453
0.563003207847778
train_acc :  0.8144  / test_acc :  0.8014
0.4848852613778382
0.5276261939751113
train_acc :  0.82765  / test_acc :  0.8141
0.4492402437672327
0.4988120390349071
train_acc :  0.8388666666666666  / test_acc :  0.8276
0.4331028675043885
0.4877254722130404
train_acc :  0.8417166666666667  / test_acc :  0.8301
0.41587146165773237
0.48077984381238803
train_acc :  0.8465  / test_acc :  0.8325
0.40298546999876594
0.46406353310167453
train_acc :  0.85195  / test_acc :  0.8369
0.3905322174365116
0.46063891928130013
train_acc :  0.8524333333333334  / test_acc :  0.8402
0.38131780274305455
0.4472351705155177
train_acc :  0.8599  / test_acc :  0.8459
0.37278104317164223
0.4660804886541499


In [11]:
network = TwoLayerNet(input_size=784, hidden_size =50,output_size=10)
iters_num=50000
train_size=x_train.shape[0]
batch_size=100
learning_rate= .1
train_loss_list=[]
train_acc_list=[]
test_acc_list=[]
iter_per_epoch = max(train_size/batch_size,1)
optimizer = Adam(0.0001)#(**optimizer_param)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch=t_train[batch_mask]
    train_loss = 0, test_loss =0
    #오차역전파
    grad = network.gradient(x_batch, t_batch)
    #갱신
    for key in {'W1', 'b1', 'W2', 'b2'}:
        optimizer.update(network.params,grad) #
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i%iter_per_epoch==0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train_acc : ",train_acc," / test_acc : ",test_acc)

KeyError: ignored